<a href="https://colab.research.google.com/github/Ianneee/Cuda_C-Cpp_into_Colab/blob/main/Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# C/C++ Cuda source code into Colab

You can run your CUDA source code in two ways: by directly uploading them in Colab to compile and execute or by using a plugin for Jupyter Notebook that enables you to write C code directly within a code cell and then run it with CUDA.

NVCC is already installed in the VM where the notebook is launched, there is no need to install any additional software.\
You can verify the installed version with:

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


## Without any plugin

1. Activate gpu runtime: click on `Runtime > Change runtime type > T4 GPU` or any other GPU if you have a paid plan.

2. Upload (or download) your source files into Colab.

In [ ]:
# Upload - more functionalities of Colab here: https://colab.research.google.com/notebooks/io.ipynb
from google.colab import files
uploaded = files.upload()

In [ ]:
# Download - Run this to try it!
!wget https://raw.githubusercontent.com/Ianneee/Cuda_C-Cpp_into_Colab/main/get_devices.cu

3. Write a `!`, for use shell commands, and after it the command for compile your downloaded file.

In [ ]:
!nvcc -Werror all-warnings get_devices.cu -o get_devices

4. Finally you can run your executable.

In [ ]:
!./get_devices

## With nvcc4jupyter plugin

1. Activate gpu runtime: click on `Runtime > Change runtime type > T4 GPU` or any other GPU if you have a paid plan.

2. Add in a code cell the following lines to download the [plugin](https://github.com/andreinechaev/nvcc4jupyter) for Jupyter Notebook and load it:

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

## Run the code

1. You can paste your source code into a Code cell adding into the first line
```
%%cu
```
to execute the cell with CUDA.

In [ ]:
%%cu
#include <cuda.h>
#include <stdio.h>

int main(){
    int deviceCount = 0;
    cudaGetDeviceCount(&deviceCount);
    if (deviceCount == 0) printf("No cuda device found\n");
    else {
        printf("Found %d devices\n", deviceCount);
        cudaDeviceProp pr;
        for (int i=0; i<deviceCount; i++){
            cudaGetDeviceProperties(&pr, i);
            printf("Device Number: %d\n", i);
            printf("  Device name: %s\n", pr.name);
            printf("  Memory Clock Rate (MHz): %d\n", pr.memoryClockRate/1024);
            printf("  Memory Bus Width (bits): %d\n", pr.memoryBusWidth);
            printf("  Peak Memory Bandwidth (GB/s): %.1f\n", 2.0*pr.memoryClockRate*(pr.memoryBusWidth/8)/1.0e6);
            printf("  Total global memory (Gbytes) %.1f\n",(float)(pr.totalGlobalMem)/1024.0/1024.0/1024.0);
            printf("  Shared memory per block (Kbytes) %.1f\n",(float)(pr.sharedMemPerBlock)/1024.0);
            printf("  minor-major: %d-%d\n", pr.minor, pr.major);
            printf("  Warp-size: %d\n", pr.warpSize);
            printf("  Concurrent kernels: %s\n", pr.concurrentKernels ? "yes" : "no");
            printf("  Concurrent computation/communication: %s\n\n",pr.deviceOverlap ? "yes" : "no");

            printf("  Max Thread per block: %d\n", pr.maxThreadsPerBlock);
            printf("  Max Thread per multiprocessor: %d\n", pr.maxThreadsPerMultiProcessor);
            printf("  Max Thread Multi processor Count: %d\n", pr.multiProcessorCount);
            printf("  Multi processor count: %d\n\n", pr.multiProcessorCount);
        }
    }
}


2. Or you can also upload (or download) your sources files into Colab in a folder named `src` and run a code cell with the mark _%%cuda_run_ in it.
The plugin will automatically find all your .cu and .h files, compile them and run.
If you don't have already run any cell with the plugin you have to create manually that folder.

In [ ]:
# Upload
from google.colab import files
uploaded = files.upload()

In [ ]:
# Download files for run the example
# Run this to try it!
![ -d src ] || mkdir src
!wget https://raw.githubusercontent.com/Ianneee/Cuda_C-Cpp_into_Colab/main/get_devices.cu
!mv get_devices.cu src

In [ ]:
%%cuda_run
# This line just to bypass an exeption and can contain any text

### More
The plugin also has other functionalities, such as the one for create a file from each single cell and then compile everything together. You can visit the [repository](https://github.com/andreinechaev/nvcc4jupyter) on github to learn more.

_Note: at 05 August 2023 %%cuda_run has a bug and can't run. I've submitted a pull request with a fix and i'm waiting for it to be accepted.
Meanwhile, you can use my fork with the fix by replacing the initial instruction with:_
```
!pip install git+https://github.com/Ianneee/nvcc4jupyter
```